In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import missingno as msno

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_validate

from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')


from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

%matplotlib inline  

housing_raw = pd.read_csv('Ames_HousePrice.csv', index_col=0)
price = housing_raw['SalePrice'].copy()
housing_raw.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [2]:
df_num = housing_raw[['GrLivArea', 'OverallQual', 'TotalBsmtSF', '1stFlrSF','GarageCars', 'GarageArea', 'YearBuilt', 'FullBath', 'YearRemodAdd', 'MasVnrArea', 'TotRmsAbvGrd', 'Fireplaces', 'BsmtFinSF1','LotFrontage','WoodDeckSF']].copy()
df_num.loc[df_num['GarageCars'].isnull(), 'GarageCars'] = 2
# df_num.loc[df_num['GarageArea'].isnull(), 'GarageArea'] = 416
df_num.loc[df_num['LotFrontage'].isnull(), 'LotFrontage'] = df_num['LotFrontage'].mean()
# df_num['LotFrontage'].fillna(df_num['LotFrontage'].mean(), inplace = True)

# Because there is only 1 null value for area, we just fill in with the Year the house was built
# df_num.loc[df_num['GarageArea'].isnull(), 'GarageYrBlt'] = df_num.loc[df_num['GarageArea'].isnull(),'YearBuilt']
df_num['GarageArea'].fillna(housing_raw[housing_raw['GarageType']== 'Detchd']['GarageArea'].mean(), inplace = True)
df_num.fillna(0,inplace = True)

In [7]:
df_cat = housing_raw.select_dtypes(['object']).copy()
df_cat.fillna('None', inplace = True)

# Define the mapping
po_ex_ordinal_mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['ExterQual'] = df_cat['ExterQual'].map(po_ex_ordinal_mapping)
df_cat['ExterCond'] = df_cat['ExterCond'].map(po_ex_ordinal_mapping)
df_cat['KitchenQual'] = df_cat['KitchenQual'].map(po_ex_ordinal_mapping)
df_cat['HeatingQC'] = df_cat['HeatingQC'].map(po_ex_ordinal_mapping)


# Defind the mapping
none_ex6_ordinal_mapping = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['BsmtQual'] = df_cat['BsmtQual'].map(none_ex6_ordinal_mapping)
df_cat['BsmtCond'] = df_cat['BsmtCond'].map(none_ex6_ordinal_mapping)
df_cat['FireplaceQu'] = df_cat['FireplaceQu'].map(none_ex6_ordinal_mapping)
df_cat['GarageQual'] = df_cat['GarageQual'].map(none_ex6_ordinal_mapping)
df_cat['GarageCond'] = df_cat['GarageCond'].map(none_ex6_ordinal_mapping)
 

# Define the mapping
none_fin_ordinal_mapping = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin':3}
# Applying the mapping
df_cat['GarageFinish'] = df_cat['GarageFinish'].map(none_fin_ordinal_mapping)


# Define the mapping
none_ex5_ordinal_mapping = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd':3, 'Ex':4}
# Applying the mapping
df_cat['PoolQC'] = df_cat['PoolQC'].map(none_ex5_ordinal_mapping)

# Define mapping
binary_mapping = {'Y': 1, 'N':0}
df_cat['CentralAir'] = df_cat['CentralAir'].map(binary_mapping)


df_cat = df_cat[['BldgType','HouseStyle','RoofStyle','MasVnrType','SaleCondition','GarageQual']].copy()

df_cat_dummy = pd.get_dummies(df_cat, drop_first=True)
df_cat_dummy

,GarageQual,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1,3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,3,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
3,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
4,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
5,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
760,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
761,3,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
762,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0


In [4]:
df = pd.concat([df_num, df_cat_dummy], axis = 1)
df

,GrLivArea,OverallQual,TotalBsmtSF,1stFlrSF,GarageCars,GarageArea,YearBuilt,FullBath,YearRemodAdd,MasVnrArea,TotRmsAbvGrd,Fireplaces,BsmtFinSF1,LotFrontage,WoodDeckSF,GarageQual,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1,856,6,856.0,856,2.0,399.0,1939,1,1950,0.0,4,1,238.0,68.516053,0,3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,1049,5,1049.0,1049,1.0,266.0,1984,2,1984,149.0,5,0,552.0,42.000000,0,3,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
3,1001,5,837.0,1001,1.0,216.0,1930,1,2007,0.0,5,0,737.0,60.000000,154,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
4,1039,4,405.0,717,1.0,281.0,1900,1,2003,0.0,6,0,0.0,80.000000,0,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
5,1665,8,810.0,810,2.0,528.0,2001,2,2001,0.0,6,0,643.0,70.000000,0,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,6,952.0,952,1.0,192.0,1916,1,1950,0.0,4,1,0.0,68.516053,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
760,1733,3,0.0,1733,2.0,452.0,1955,2,1955,0.0,8,1,0.0,68.516053,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
761,2002,5,1001.0,1001,3.0,871.0,1949,2,1950,0.0,8,0,284.0,82.000000,0,3,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
762,1842,7,985.0,985,2.0,486.0,2000,2,2000,144.0,7,1,841.0,68.516053,193,3,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0


In [5]:
x = df
y = price

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=42)

In [25]:
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [140, 141, 150],
    'min_samples_split':[12, 15, 20],
    'max_depth': [30, 35, 40]

}
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [30, 35, 40],
                         'min_samples_split': [12, 15, 20],
                         'n_estimators': [140, 141, 150]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
best_forest = grid_search.best_estimator_
print(best_forest)
best_forest.score(x_test, y_test)

RandomForestRegressor(max_depth=30, min_samples_split=15, n_estimators=141)


0.8814859546459342

In [40]:
# train_data = x_train.join(y_train)

In [65]:
# plt.figure(figsize=(25, 15))
# sns.heatmap(train_data.corr(), annot = True, cmap='coolwarm')


In [11]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)
print('Train score:', rf.score(x_train,y_train))
print('Test score:', rf.score(x_test,y_test))

Train score: 0.9826097731029175
Test score: 0.8962430585334156


In [9]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)

RandomForestRegressor()

In [10]:
rf.score(x_train,y_train)

0.9839793485739693

In [11]:
rf.score(x_test,y_test)

0.889588470853166

In [93]:
# # Create the preprocessing pipelines for both numeric and categorical data

# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first'))  # This will one-hot encode the categorical features
# ])

# # Combine the numeric and categorical transformations
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', categorical_transformer, cat_cols)
#     ])

# # Create a pipeline that combines the preprocessor with a regression model
# lasso_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', Lasso())
    
# ])

# ridge_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', Ridge())
    
# ])

# rf_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', ensemble.RandomForestRegressor())
    
# ])
# rf_pipeline

In [ ]:
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score:", best_score)

In [90]:
# cat_cols = df.select_dtypes(include=['object']).columns
# num_cols = df.select_dtypes(include = ['int64','float64']).columns
# print('Num of cat cols:', len(cat_cols))
# print('Num of num cols:', len(num_cols))

Num of cat cols: 6
Num of num cols: 16
